# Recommender_System

We can take into account two examples. Content Based and Collaborative Filtering(CF).

- Colaborative filtering produces recomendations based on the knowledge of users attitude to items. 

- Content based focuses on attributes of items. Later gives you reccomendations based on similarity between them. 

The algorithm has the ability to learn on it's own. Meaning it can decide what features to use. 
Spark.ml uses the alternating least squares(ALS) algorithm to know these latent factors. 
Another words it can fulfill missing data on it's own. It uses the specific format to do it. So data needs to be properly prepared. To sum it pup. The ALS uses crowd wisdom to predict further items. 

In [27]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommender_sys').getOrCreate()

In [29]:
from pyspark.ml.recommendation import ALS

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator

In [31]:
df = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)

In [32]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [33]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [34]:
train_df, test_df = df.randomSplit([0.8, 0.2])

In [35]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [36]:
model = als.fit(train_df)

In [37]:
predictions = model.transform(test_df)

In [38]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      0|   3.0|    28|0.45821905|
|      1|   1.0|    28|-3.8295836|
|      2|   4.0|    28|  2.577623|
|      1|   1.0|    26| 0.3426708|
|      2|   1.0|    26|  2.076768|
|      4|   2.0|     1| 1.1779118|
|      3|   1.0|    13| 0.4558654|
|      6|   1.0|    13| 1.3460605|
|      5|   1.0|     6| 1.9306772|
|      2|   2.0|    20| 3.1863234|
|      0|   1.0|    19| 0.5484833|
|      2|   1.0|    19| 3.2078025|
|      0|   1.0|    15| 1.4135474|
|      6|   1.0|     9| 1.0774016|
|      4|   1.0|    23| 0.6385886|
|      3|   1.0|     7| 0.8519964|
|      4|   1.0|     7| 2.2586262|
|      4|   1.0|    24| 1.6213839|
|      4|   1.0|    29| 1.9470978|
|      0|   1.0|    21|0.21145405|
+-------+------+------+----------+
only showing top 20 rows



In [39]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [40]:
rmse = evaluator.evaluate(predictions)

In [41]:
print('RMSE')
print(rmse)

RMSE
1.753159799259631


In [42]:
single_user = test_df.filter(test_df['userId']==11).select(['movieId', 'userId'])

In [43]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     37|    11|
|     43|    11|
|     66|    11|
|     71|    11|
|     76|    11|
|     81|    11|
|     88|    11|
|     97|    11|
|     99|    11|
+-------+------+



In [46]:
recommendations = model.transform(single_user)
recommendations.show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     81|    11| -0.6248032|
|     76|    11|   2.826518|
|     43|    11|   2.526423|
|     37|    11| -2.0028026|
|     88|    11|-0.44395733|
|      9|    11|  0.5848503|
|     97|    11| -0.4232418|
|     71|    11|-0.59483933|
|     99|    11|  -0.379709|
|     66|    11|0.115451574|
+-------+------+-----------+



In [54]:
from pyspark.sql.functions import *

In [61]:
recommendations.orderBy(desc('prediction')).head(10)

[Row(movieId=76, userId=11, prediction=2.8265180587768555),
 Row(movieId=43, userId=11, prediction=2.5264229774475098),
 Row(movieId=9, userId=11, prediction=0.5848503112792969),
 Row(movieId=66, userId=11, prediction=0.11545157432556152),
 Row(movieId=99, userId=11, prediction=-0.37970900535583496),
 Row(movieId=97, userId=11, prediction=-0.4232417941093445),
 Row(movieId=88, userId=11, prediction=-0.4439573287963867),
 Row(movieId=71, userId=11, prediction=-0.594839334487915),
 Row(movieId=81, userId=11, prediction=-0.6248031854629517),
 Row(movieId=37, userId=11, prediction=-2.002802610397339)]